# Server Running


In [1]:
from flask import Flask, request, jsonify
import os
from datetime import datetime

app = Flask(__name__)
LOG_FILE = "sensor_data_log.txt"

@app.route('/api/sensor', methods=['POST'])
def receive_sensor_data():
    try:
        data = request.get_json()

        # Ensure data is not None and contains expected fields
        if not data:
            raise ValueError("No JSON data received or JSON is empty.")
        
        # Extract data with .get() for safety, providing default values if keys are missing
        # Removed 'light' and 'pressure' as per ESP32 sender's current payload
        rfid = data.get('rfid', 'N/A')
        sex = data.get('sex', 'N/A') # Added sex field from the incoming JSON
        weight = data.get('weight', 'N/A')
        temperature = data.get('temperature', 'N/A')
        humidity = data.get('humidity', 'N/A')

        # Format the data as a log entry
        timestamp_log = datetime.now().strftime("%Y-%m-%d %H:%M:%S") # Use a different variable name to avoid confusion
        log_entry = (
            f"[{timestamp_log}]\n"
            f"RFID: {rfid}\n"
            f"Sex: {sex}\n" # Include sex in the log
            f"Weight: {weight} kg\n"
            f"Temperature: {temperature} °C\n"
            f"Humidity: {humidity} %\n"
            f"{'-'*40}\n"
        )

        # Append to log file
        with open(LOG_FILE, "a") as f:
            f.write(log_entry)

        print("✅ Data received and logged.")
        # Print the actual incoming JSON for debugging on the server side
        print("Incoming JSON:", data)
        return jsonify({"message": "Data logged successfully"}), 200

    except Exception as e:
        print("❌ Error processing data:", e) # More descriptive error message
        # Log the raw request data for debugging if an error occurs
        try:
            print("Raw request data:", request.data.decode('utf-8'))
        except Exception as decode_e:
            print("Could not decode raw request data:", decode_e)
        return jsonify({"error": "Failed to process data", "details": str(e)}), 400 # Include exception details

if __name__ == '__main__':
    # It's good practice to run Flask in debug mode for development,
    # but turn it off for production.
    app.run(host='0.0.0.0', port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.144:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\Zwivh\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
